# Lucas Pierru - PIEL14069708 - Équipe 16 - Laboratoire 3 GPA671

# Exercice 1

## Question 1

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from sklearn import datasets
X, y = datasets.make_blobs(n_samples=[100, 100, 1000], cluster_std=[1.0, 2.5, 0.5], random_state=17)
# On sépare des points en trois clusters. n_samples correspond au nombre de points dans chaque clusters
# cluster_std correspond à l'écart entre chaque point.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

## Question 2

### Question A

In [ ]:
def random_init(X_train, k):
    rands = np.random.choice(len(X_train), k, replace = False) # On choisit aléatoirement 3 points 
                                                               # parmis tous ceux disponibles
    return X_train[rands] # On retourne ces points comme étant nos centres

In [ ]:
for i in range (10):
    rand_center = random_init(X,3)
    plt.scatter(rand_center[:,0], rand_center[:,1], marker = i) #Bleu, vert, jaune, rouge, violet


On voit que les centres sont souvent en haut à droite du graphique et donc proche du 3ème cluster. Ceci est logique vu que nous avons 1000 données pour ce cluster contre 100 pour chacun des autres et donc comme le choix des points est aléatoire, on a plus de chance de tomber sur un point de ce cluster là.

### Question B

In [ ]:
def kmeans_plusplus(X_train,k):
    centers = np.zeros((k,len(X_train[0])))
    rand = np.random.choice(len(X_train), 1) # On choisit le premier point aléatoirement
    centers[0] = X_train[rand][0] # On le définit comme notre étant notre premier centre

    for i in range(1,k):
        distance = X_train - np.reshape(centers,(k,1,len(X_train[0])))
        distances = np.sqrt(np.sum((distance)**2, axis=2)) # On calcul les distances avec les centres        
        min_dist = np.min(distances[:i],axis=0) # On gardes les distances minimales
        prob = min_dist/np.sum(min_dist) # On calcul la probabilité en fonction de la distance
        rand = np.random.choice(len(X_train), 1, p = prob) # On choisit notre nouveau centre aléatoirement avec celle-ci
        centers[i] = X_train[rand]
    return centers

In [ ]:
for i in range (5):
    rand_center = kmeans_plusplus(X,3)
    plt.scatter(rand_center[:,0], rand_center[:,1], marker = i) #Bleu, vert, jaune, rouge, violet

On peut voir que les centres sont mieux répartis dans les différents clusters. Cela n'est pas parfait mais cet initialisation semble meilleur que l'initialisation aléatoire.

## Question 3

In [ ]:
class K_moyennes:
    def __init__(self, k:int, init_choice:bool) -> None:
        """
        Classificateur K-moyennes.
        
        Parameters
        ----------
        k: int
            Paramètre du nombre de clusters que l'on veut avoir.
        init_choice : bool
            Paramètre du choix du type d'initialisation.
        """
        self.k = k
        self.init_choice = init_choice #0 : rand ; 1 : kmeans_plusplus

    def fit(self, X_train:np.ndarray) -> None:
        """
        Entrainement de l'algorithme K-moyennes.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# points, ).
        """
        self.X_train = X_train
        
        # On choisit notre initialisation 0 = random ; 1 = kmeans++

        if self.init_choice :
            self.centers = kmeans_plusplus(self.X_train, self.k)
        else:
            self.centers = random_init(self.X_train, self.k)
        
        self.X_train = np.reshape(self.X_train,(len(self.X_train),1,len(X_train[0]))) # On change la forme pour pouvoir 
                                                                                      # Broadcaster les données
        
        self.J_array =  []
        self.J = 0
        previous_J = 1
        margin = 0.01

        while (self.J>previous_J+margin) | (self.J<previous_J-margin):
            self.norm_pc = (self.norm(self.X_train, self.centers))**2 # On calcul la norme au carré des données d'entrainement
                                                                      # moins les centres
            self.argmin = np.argmin(self.norm_pc, axis = 1) # On prend l'argument du minimum pour chaque donnée
            previous_J = self.J
            self.J = np.sum(np.min(self.norm_pc, axis=1)) # On calcul J
            sum_x = np.zeros((self.k,len(X_train[0])))  
            sum_arg = np.zeros(self.k)

            # On calcul nos nouveaux centres

            for i in range(self.k):
                sum_x[i,:] = np.sum(self.X_train[np.where(self.argmin == i)],axis=0)
                sum_arg[i] = np.sum((self.argmin == i)*1)
            self.J_array.append(self.J)
            self.centers = sum_x/np.reshape(sum_arg,(self.k,1))

    def fit_restart(self, n_restart:int, X_train:np.ndarray) -> None:
        """
        Entrainement de K-moyennes avec restart.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# exemples,).
        n_restart : int
            Nombre de fois que l'on recommence l'entrainement.

        """
        J_min = 10**8
        for n in range (n_restart):
            self.fit(X_train)
            if self.J < J_min:
                J_min = self.J
                self.centers_restart = self.centers
                self.J_array_restart = self.J_array
        self.centers = self.centers_restart
        self.J_array = self.J_array_restart

    def predict(self, X_val:np.ndarray) -> np.ndarray:
        """
        Prédiction du cluster en fonction de la distance euclidienne d'un point.
        
        Parameters
        ----------
        X_val : np.ndarray
            Données de validation. Taille : (# exemples, ).
        """
        self.X_val = X_val
        self.X_val = np.reshape(self.X_val,(len(self.X_val),1,len(self.X_val[0])))
        self.diff = self.X_val - self.centers
        self.distance = np.sqrt(np.sum((self.diff**2),axis=2)) # On calcul la distance euclidienne des points
                                                               # par rapport à chaque centres.
        return np.argmin(self.distance, axis = 1)

    def norm(self, X, U) -> np.ndarray:
        return np.sqrt(np.sum((X-U)**2,axis=2))

    def compress_image(self, X_val):
        pred = self.predict(X_val)
        
        return self.centers[pred]


## Question 4

In [ ]:
# On crée 2 objets avec les deux initialisations.

km0 = K_moyennes(3, 0)
km0.fit(X)
km1 = K_moyennes(3, 1)
km1.fit(X)

y_pred0 = km0.predict(X)
y_pred1 = km1.predict(X)

# On affich les données avec les deux initialisations

plt.rcParams["figure.figsize"] = (6,8)
plt.figure(1)
plt.subplot(211)
plt.scatter(X[:,0], X[:,1], c=y_pred0, s=30, cmap=plt.cm.Paired)
plt.scatter(km0.centers[:,0], km0.centers[:,1], color = 'blue')
plt.title('Random')
plt.subplot(212)
plt.scatter(X[:,0], X[:,1], c=y_pred1, s=30, cmap=plt.cm.Paired)
plt.scatter(km1.centers[:,0], km1.centers[:,1], color = 'blue')
plt.title('Kmeans++')

In [ ]:
# On affiche la courbe de J en fonction de l'initialisation.

plt.plot(km0.J_array, color = 'blue', label='Random')
plt.plot(km1.J_array, color = 'red', label='Kmeans++')
plt.yscale('log')
plt.title("Valeurs de J en fonction de l'initialisation")
plt.legend()


L'initialisation aléatoire est moins bonne que celle de kmeans_plusplus car on arrive moins souvent à converger avec celle-ci

## Question 5

In [ ]:
km = K_moyennes(3, 1)
km.fit(X)
km_restart = K_moyennes(3, 1)
km_restart.fit_restart(5,X) 
y_pred_restart = km_restart.predict(X)

plt.rcParams["figure.figsize"] = (6,8)
plt.figure(1)
plt.subplot(211)
plt.scatter(X[:,0], X[:,1], c=y_pred0, s=30, cmap=plt.cm.Paired)
plt.scatter(km.centers[:,0], km.centers[:,1], color = 'blue')
plt.title('No restart')
plt.subplot(212)
plt.scatter(X[:,0], X[:,1], c=y_pred1, s=30, cmap=plt.cm.Paired)
plt.scatter(km_restart.centers[:,0], km_restart.centers[:,1], color = 'blue')
plt.title('Restart')

On peut voir que fit_restart nous permet d'avoir des bons centres et donc la convergence dès la première exectution.

In [ ]:
from skimage.data import chelsea, astronaut, coffee

# On normalise les données

chelsea = chelsea().astype(np.float32) / 255
astronaut = astronaut().astype(np.float32) / 255
coffee = coffee().astype(np.float32) / 255

In [ ]:
# On crée une fonction qui va compresser les images
def compress_image(image, k, nb_restart):
    plt.axis("off")
    plt.imshow(image)
    plt.show()

    y_len = len(image)
    x_len = len(image[0])
    z_len = len(image[0][0])
    # On reshape l'image pour pouvoir faire l'entrainement.
    image_reshaped = np.reshape(image, (y_len*x_len, z_len))

    km_image = K_moyennes(k, 1)
    km_image.fit_restart(nb_restart, image_reshaped)
    image_compressed = km_image.compress_image(image_reshaped)
    # On reshape dans le format original
    image_compressed = np.reshape(image_compressed, (y_len, x_len, z_len))

    plt.axis("off")
    plt.imshow(image_compressed)
    plt.show()

In [ ]:
compress_image(chelsea,5,5)


In [ ]:
compress_image(coffee, 5, 5)

In [ ]:
compress_image(astronaut, 5, 5)

On peut voir que la dernière image est la moins approprié pour cet algorithme. En effetm les nuances de gris en arrière plan de la photo semble perturber les résultats et compromettre l'image obtenue.

# Exercice 2

## Question 1

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split

plt.rcParams["figure.figsize"] = (4,4)
n = 5

# On récupére les données d'entrainement et les étiquettes.

rand_faces_X  = fetch_olivetti_faces(return_X_y=True)[0]
rand_faces_y  = fetch_olivetti_faces(return_X_y=True)[1]

rand = np.random.choice(len(fetch_olivetti_faces(return_X_y=True)[0]), n,replace = False)

rand_faces_images = fetch_olivetti_faces().images[rand]

# On affiche n images aléatoire.

for i in range(n):
    plt.imshow(rand_faces_images[i], cmap=plt.cm.gray)
    plt.show()

X_train, X_val, y_train, y_val = train_test_split(rand_faces_X, rand_faces_y, test_size = 0.6, stratify=rand_faces_y)

## Question 2

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

Gaussian_model = GaussianNB()

# On crée un classificateur naïf de bayes pour classifier nos données

Gaussian_model.fit(X_train, y_train)
Gaussian_score = Gaussian_model.score(X_val,y_val)
print("Score pour le classificateur Bayésien :",round(Gaussian_score*100,2),"%")

## Question 3

In [ ]:
X_moy = np.mean(X_train, axis=0)
X_center = X_train-X_moy
C = np.transpose(X_center)@X_center # On calcul la matrice de covariance.


In [ ]:
lambd, rho = np.linalg.eigh(C) # On calcul nos vecteurs et valeurs propres.

# On classe les valeurs propres en ordre croissant et on classe nos vecteurs correspondant.
# On calcul également, la somme cumulée des valeurs propres.

lambd_norm = np.linalg.norm(lambd)
lambd = lambd[::-1]
lambd_norm_sum = np.cumsum(lambd/lambd_norm)
rho = rho[:,::-1]
plt.plot(lambd_norm_sum, color = 'blue', label='Random')
plt.title("Somme cummulée des valeurs propres normalisées")

La somme cumulée des valeurs propres normalisées correspond à la valeur de reconstruction de l'image et nous permet de deteriner le nombre de vecteur nécessaires. Dans notre exemple on peut voir que cette somme cumulée arrive à son maximum aux alentours de 200 vecteurs et donc avec ces 200 vecteurs (parmis 4096) on peut reconstruire l'image correctement.

## Question 4

In [ ]:
for i in range(10):
    plt.imshow(np.reshape(rho[:,i],(64,64)), cmap=plt.cm.gray)
    plt.show()

## Question 5

In [ ]:
nb_vect = 200

# On projette nos données et on reconstruit les images.

Z = X_center @ rho[:,:nb_vect]
X_reconstruit = Z @ np.transpose(rho[:,:nb_vect]) + X_moy

m = 8
rand_val = np.random.choice(len(X_reconstruit), m,replace = False)
X_reconstruit_rand = X_reconstruit[rand_val]

for i in (rand_val):
    plt.imshow(np.reshape(X_train[i],(64,64)), cmap=plt.cm.gray)
    plt.show()

for j in range(m):
    plt.imshow(np.reshape(X_reconstruit_rand[j],(64,64)), cmap=plt.cm.gray)
    plt.show()


Plus on augmente le nombre de vecteurs plus on va se retrouver proche de l'image originale et inversement. Avec un nombre très faible de vecteur, les images reconstruite se ressemble plus ou moins toutes.

## Question 6

In [ ]:
Gaussian_model_reconstruit = GaussianNB()

# On recrée un classificateur naïf bayesien avec les données reconstruites

nb = 10
Z = X_center @ rho[:,:nb]
X_reconstruit = Z @ np.transpose(rho[:,:nb]) + X_moy

Gaussian_model_reconstruit.fit(X_reconstruit, y_train)
Gaussian_score = Gaussian_model_reconstruit.score(X_val,y_val)
print("Score pour le classificateur Bayésien :",round(Gaussian_score*100,2),"%", "Pour",nb,"vecteurs.")

La précision est très mauvaise pour 10 vecteurs, on est nettement inférieure à la valeur obtenu à la question 2

## Question 7

In [ ]:
import time

start = time.time()
Gaussian_model_reconstruit = GaussianNB()
nb = 145
Z = X_center @ rho[:,:nb]
X_reconstruit = Z @ np.transpose(rho[:,:nb]) + X_moy

Gaussian_model_reconstruit.fit(X_reconstruit, y_train)
Gaussian_score = Gaussian_model_reconstruit.score(X_val,y_val)
end = time.time()
print("Temps de calcul :",round(end - start,2),"secondes")
print("Score pour le classificateur Bayésien :",round(Gaussian_score*100,2),"%", "Pour",nb,"vecteurs.")

En choisissant d'autres nombres de vecteurs, on peut augmenter la précision obtenu. On peut au maximum se retrouver à 70% de précision ce qui est mieux que ce qu'on a obtenu jusque là. Plus on augmente le nombre de vecteurs plus le temps de calcul sera long. Pour 145 vecteurs, on obtient un temps de calcul de 1.4s environ.

## Question 8

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# On fait la même chose avec un LDA.

start = time.time()

lda = LinearDiscriminantAnalysis()
lda.fit(X_reconstruit, y_train)
lda_score = lda.score(X_val,y_val)

end = time.time()
print("Temps de calcul :",round(end - start,2),"secondes")

print("Score pour le LDA :",round(lda_score*100,2),"%")

Avec le LDA, on obtient un taux d'exactitude bien supérieur à ce qu'on avait avant (le LDA est meilleur d'environ 20%)ainsi qu'un temps d'inférence énormément réduit, le temps de calcul obtenu est de 0.09s. Le LDA est donc une meilleure solution pour notre problème.